In [1]:
from scipy import ndimage
from skimage import io,util,color
from IPython import display
import numpy
import sys
import math

import ipywidgets as widgets
from ipywidgets import interactive
from IPython.display import display, clear_output

import matplotlib.pyplot as plt

# Our imports
import skimage
import scipy
from skimage import util,color,exposure,io
from scipy import fftpack

%matplotlib inline

In [2]:
def readAndShowImage(image):
    im = io.imread(image, as_gray=True)
    io.imshow(im, cmap='gray')
    io.show()

inputImage = widgets.Dropdown(
    options={
        'Saturn':'../Bilder/image_resources/suaturn-rings-sinusoidal-interf.tif',
        'White bar':'../Bilder/white_bar.png',
        'White bar translate':'../Bilder/white_bar_translated.png',
        'white bar rotated':'../Bilder/white_bar_rotated.png',
        'test pattern':'../Bilder/test_pattern.pgm'
        },
    description='Bilde: ',
    disabled=False,
)

interactive(readAndShowImage, image=inputImage)

interactive(children=(Dropdown(description='Bilde: ', options={'Saturn': '../Bilder/image_resources/suaturn-ri…

In [3]:
def dist(i,j,width,height):
    center_i, center_j = (height/2,width/2)
    return ((i-center_i)**2 + (j-center_j)**2)**0.5

def fourrier(image,cutoff):
    # Read image, convert to gray, convert to float
    im = util.img_as_float(color.rgb2gray(io.imread(image)))

    # Retrieve the image dimensions
    height,width = im.shape
    
    #finding P and Q
    padded_height = height*2
    padded_width = width*2
    
    #Padding image width 0s
    padded = numpy.zeros((padded_height,padded_width))
    padded[0:height,0:width] = im

    # Create an image to store the transformed image
    image_trans = numpy.zeros(padded.shape)
    # Transform each pixel in the image by multiplying with -1**(i+j) (see 
    # slides/book) to translate the fourier transformed image to height/2 width/2 
    # i.e. the center of the image
    for i in range(padded_height):
      for j in range(padded_width):
        image_trans[i,j] = padded[i,j]*((-1)**(i+j))

    # Perform the fourier transformation
    fourier_transform = fftpack.fft2(image_trans)



    #lage filter 
    filtr = numpy.zeros(padded.shape)
    for x in range(padded_width):
        for y in range(padded_height):
            filtr[x,y] = 0 if dist(x,y,padded_width,padded_height) < cutoff else 1
    
    #filtrerer den fourier transformerte
    filtered = filtr*fourier_transform    
    
    #transformerer med invers fourier
    filtered = fftpack.ifft2(filtered)
    
    #translerer tilbake
    filtered_translated = numpy.zeros(padded.shape)

    for i in range(padded_height):
        for j in range(padded_width):
            filtered_translated[i,j] = filtered[i,j]*((-1)**(i+j))
    
    #fjerner padding
    filtered_cropped = filtered_translated[0:width,0:height]
    
    # Show the result
    io.imshow(numpy.clip(filtered_cropped,0,1), cmap='gray')
    io.show()
    
interactive(fourrier,image=inputImage,cutoff=widgets.IntSlider(min=1,max=300,step=1,value=50,continuous_update=False,))

interactive(children=(Dropdown(description='Bilde: ', options={'Saturn': '../Bilder/image_resources/suaturn-ri…